# Workshop Notebook 3: Text Clustering on NECTEC's Thai QA Dataset



In this notebook, we will cluster type of Thai questions by its semantic similarity.


We will use data from Thai QA Dataset (https://www.nectec.or.th/corpus/index.php?league=tp5).

In [ ]:
!pip install --user -q --upgrade --pre pythainlp

### Header

In [1]:
# Header

import os
import json
from collections import Counter
import numpy as np
from pythainlp.tokenize import word_tokenize


### 1. Load Question Answeing Dataset


__Thai Question Answering Dataset from Wikipedia created by NECTEC__

Reference: http://copycatch.in.th/thai-qa-task.html

ชุดข้อมูลนี้ประกอบด้วยกลุ่มคู่คำถามและคำตอบที่ถูกสร้างจากผู้ใช้ทั่วไปและเป็นกลุ่มคู่คำถามและคำตอบที่มีเนื้อหาหลากหลาย เช่น ด้านวิทยาศาสตร์ การท่องเที่ยว กีฬา และอื่น ๆ นอกจากนี้คำถามที่อยู่ในกลุ่มนี้เป็นคำถามง่ายและยากผสมกัน 

คำถามที่เป็นข้อเท็จจริง (Factoid question answering) เป็นคำถามที่ง่ายๆ ถามเกี่ยวกับข้อเท็จจริงและเป็นคำถามที่มีคำตอบจริง โดยจะมีคำแสดงคำถามได้แก่ ใคร อะไร ไหน ที่ไหน เมื่อไร ใด กี่ เท่าไร ยกตัวอย่างเช่น

-----

__คำถาม:__ นายกรัฐมนตรีคนที่ 7 ของประเทศไทยคือใคร 

__คำตอบ:__ ปรีดี พนมยงค์

-----

__คำถาม:__ กีฬาประจำชาติแห่งแดนอาทิตย์อุทัยที่มีประวัติยาวนานคือกีฬาอะไร 

__คำตอบ:__ ซูโม่

-----

__คำถาม:__ ออสเตรเลียเป็นประเทศร่ำรวยเป็นอันดับที่เท่าไรของโลก

__คำตอบ:__ 12

-----

__คำถาม:__ พระกระโดดกำแพงเป็นอาหารประจำชาติไหนในทวีปเอเชีย 

__คำตอบ:__ จีน

-----

__คำถาม:__ รายงานความสุขโลกเป็นดัชนีวัดความสุขเผยแพร่โดยใคร 

__คำตอบ:__ สหประชาชาติ

In [2]:
# Download dataset

dataset = None

with open("../data/thai_qa/ThaiQACorpus-DevelopmentDataset.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)["data"]

In [3]:
print("Total Number of questions:", len(dataset))

Total Number of questions: 4000


In [4]:
for qa in dataset[:10]:
    print(qa)
    print("")

{'question_id': 1, 'question': 'สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร', 'answer': 'ฮิกกิ้นส์', 'answer_begin_position ': 529, 'answer_end_position': 538, 'article_id': 115035}

{'question_id': 2, 'question': 'ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวียต มีชื่อเรียกอีกชื่อว่าอะไร', 'answer': 'เมชตา', 'answer_begin_position ': 139, 'answer_end_position': 144, 'article_id': 376583}

{'question_id': 3, 'question': 'ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวียต สามารถสังเกตและตรวจวัดอะไรได้โดยตรง', 'answer': 'ลมสุริยะ', 'answer_begin_position ': 641, 'answer_end_position': 649, 'article_id': 376583}

{'question_id': 4, 'question': 'กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอรมนี ก่อตั้งขึ้นในปี ค.ศ. ใด', 'answer': 'ปี 1933', 'answer_begin_position ': 304, 'answer_end_position': 311, 'article_id': 547560}

{'question_id': 5, 'question': 'ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบดีคนใดของสหรัฐอเมริกา', 'answer': 'ประธานาธิบดีวิลเลีย

### Question words 

__Wh-question:__ `What, When, Where, Why, Which, How`

- In Thai questions, it may occur at the end of sentence, or at the begining.


For example:



- "__ใครคือ__ผู้บริหารสำนักงานส่งเสริมเศรษฐกิดิจิทัล" -> Who is the director of Digital Economy Promotion Agency?
- "แม่ของดอนัลด์ ทรัมป์__คือใคร__" -> Who is the mother of Donald Trump?
- "บารัค โอบามา มีเชื่อชาติ__อะไร__" -> What is the race of Barack Obama?
- "ดอนัลด์ ทรัมป์ เกิด__เมื่อใด__" -> When was Donald Trump born?
- "ทะเลเดดซี __ตั้งอยู่ที่ใด__" -> Where is the Dead Sea located?
- "คุณอาศัยอยู่ที่__ประเทศใด__" -> Which country do  you live in?



__Look at the last 3 tokens and fist 3 tokens__

In [5]:
c = Counter()
for qa in dataset[1000:1025]:
    question = qa["question"]
    print("Question:", question)
    toks = word_tokenize(question, keep_whitespace=False)
    print("Tokens:", "|".join(toks))
    print("First 3 Tokens:", "|".join(toks[:3]))
    print("Last 3 Tokens:", "|".join(toks[-3:]))
    print("")

Question: ปู่ของเอมี่ กลิ่นประทุม หรือ มีชื่อจริงว่า เอมิกา กลิ่นประทุม คือใคร
Tokens: ปู่|ของ|เอ|มี่|กลิ่น|ประ|ทุม|หรือ|มี|ชื่อจริง|ว่า|เอ|มิ|กา|กลิ่น|ประ|ทุม|คือ|ใคร
First 3 Tokens: ปู่|ของ|เอ
Last 3 Tokens: ทุม|คือ|ใคร

Question: ญาญ่า อุรัสยา เสปอร์บันด์ ได้เข้าร่วมโครงการค้นหานางแบบหน้าใหม่ โดยคำชักชวนของใคร
Tokens: ญาญ่า|อุ|รัส|ยา|เส|ปอ|ร์|บัน|ด์|ได้|เข้าร่วม|โครงการ|ค้นหา|นางแบบ|หน้าใหม่|โดย|คำชักชวน|ของ|ใคร
First 3 Tokens: ญาญ่า|อุ|รัส
Last 3 Tokens: คำชักชวน|ของ|ใคร

Question: แอน ทองประสม แสดงละครเรื่องแรกกับสถานีโทรทัศน์ไทยทีวีสีช่อง 3 ปี 2535 ในเรื่องอะไร
Tokens: แอน|ทอง|ประสม|แสดงละคร|เรื่อง|แรก|กับ|สถานีโทรทัศน์|ไทย|ทีวีสี|ช่อง|3|ปี|2535|ใน|เรื่อง|อะไร
First 3 Tokens: แอน|ทอง|ประสม
Last 3 Tokens: ใน|เรื่อง|อะไร

Question: หยาดทิพย์ ราชปาล มีชื่อเล่นว่าอะไร
Tokens: หยาด|ทิพย์|ราช|ปาล|มี|ชื่อเล่น|ว่า|อะไร
First 3 Tokens: หยาด|ทิพย์|ราช
Last 3 Tokens: ชื่อเล่น|ว่า|อะไร

Question: พระเจ้าวรวงศ์เธอ พระองค์เจ้าโสมสวลี พระวรราชาทินัดดามาตุ พระนามเดิมว่าอะไร
Tokens: พระเจ้าวรวงศ์

### 2. Sentence Vectorization
Use Sentence Vectorizer to represent a sentence as a 300 dimensional vector

In [6]:
!pip install gensim

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

from pythainlp.word_vector import similarity, sentence_vectorizer

#### 2.1 Calculate Cosine similarility


![title](images/cosine_sim.png)
image from https://cmry.github.io/notes/euclidean-v-cosine

In [8]:
def caluculate_consine_similarity(sentence_a, sentence_b):
    vec1, vec2 = sentence_vectorizer(sentence_a),sentence_vectorizer(sentence_b)
    return cosine_similarity(vec1, vec2)[0][0]

In [9]:
caluculate_consine_similarity("ทานอาหาร", "กินข้าว")

0.836088698739909

In [10]:
caluculate_consine_similarity("กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอรมนี ก่อตั้งขึ้นในปี ค.ศ. ใด",
                              "กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม ในประเทศไทย ก่อตั้งในปี พ.ศ. ใด")

0.7311166186804937

In [11]:
caluculate_consine_similarity("กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอรมนี ก่อตั้งขึ้นในปี ค.ศ. ใด",
                              "สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji มีชื่อว่าอะไร")

0.44678680188394576

#### __Try out:__ Add you own sentences or words.

In [12]:
caluculate_consine_similarity("สุนัข", "แมว")

0.709938775067541

In [13]:
caluculate_consine_similarity("สุนัข", "นาฬิกาทราย")

0.032584644862714035

caluculate_consine_similarity("สุนัข", "แมว")

#### 2.2 Vectorize all questions in the dataset

In [14]:
question_feat = np.array([ sentence_vectorizer(qa["question"]) for qa in dataset ])

In [15]:
question_feat = question_feat.reshape(len(dataset), -1)
question_feat.shape

(4000, 300)

__Save as .tsv file__

In [16]:
import io

out_v = io.open('./qa/vector.tsv', 'w', encoding='utf-8')
out_m = io.open('./qa/meta.tsv', 'w', encoding='utf-8')

for index, qa in enumerate(dataset):
    question = qa["question"]
    vector = question_feat[index]
    out_m.write(question + "\n")
    out_v.write('\t'.join([str(x) for x in vector]) + "\n")
out_v.close()
out_m.close()

## 3. Visualize sentence embeddings



#### __1. Go to: http://projector.tensorflow.org/__





#### __2. Click "Load" button on the left panel__

![title2](images/projector_left_panel.png)




#### __3. Then, Upload `vector.tsv`, and `meta.tsv`__

![title3](images/projector_load_data.png)




#### __4. Select T-SNE, the screen will show the embedding vector for each sentence in 3D space.__

![title1](images/projector_main.png)


## 4. Text Clustering with K-Mean Clustering


In [17]:
from sklearn.cluster import k_means

In [18]:
N_CLUSTERS = 7

In [19]:
question_texts = []
for qa in dataset:
    tokens = word_tokenize(qa["question"], keep_whitespace=False)
    question_text = ''.join(tokens)
    question_texts.append(question_text)

In [20]:
question_texts[0]

'สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่องBenjiที่ออกฉายในปีพ.ศ.2517มีชื่อว่าอะไร'

In [21]:
question_feats = np.array([ sentence_vectorizer(question_text) for question_text in question_texts])

In [22]:
question_feats = question_feats.reshape(len(dataset), -1)
question_feats.shape

(4000, 300)

In [23]:
def build_clusters(n_clusters):
    centroids, labels, _ = k_means(question_feats, n_clusters=n_clusters)
    return centroids, labels

def explore_cluster(n_clusters, centroids, labels, n_qas=10):
    clusters = { i: [] for i in range(0, N_CLUSTERS, 1) }
    for index, label in enumerate(labels):
        clusters[label].append( dataset[index]["question"])
    
    for cluster_id, members in clusters.items():

        print("Cluster ID:", cluster_id)
        print("Number of questions in this cluster:", len(members))
        for i, member in enumerate(members[100:100+n_qas]):
            print("")
            print((i+1), "", member)
        print("\n\n")
    

In [24]:
centroids, labels = build_clusters(7)
explore_cluster(7, centroids, labels, 10)

Cluster ID: 0
Number of questions in this cluster: 620

1  หลู่ ซฺวิ่น เป็นนามปากกาของนักเขียนชาวจีนที่สำคัญคนหนึ่งของคริสต์ศตวรรษที่ 20 คือใคร

2  โจว จางโช่ว เป็นนักเขียนชาวจีนที่สำคัญคนหนึ่งของคริสต์ศตวรรษที่ 20 ใช้นามปากกาว่า หลู่ ซฺวิ่น เริ่มเข้าเรียนที่โรงเรียนอะไร

3  องค์บาก 2 เป็นภาพยนตร์ไทยเน้นศิลปะการต่อสู้ ที่นำแสดงโดยใคร

4  เศษชิ้นส่วนขนาดเล็กของดาวหางที่ถูกสลัดทิ้งไว้ตามทางโคจรเรียกว่าอะไร

5  วากิ้นที่เป็นปลาทองสายพันธุ์หนึ่งมีแหล่งกำเนิดที่ประเทศใด

6  กระสวยอวกาศดิสคัฟเวอรีเป็นกระสวยอวกาศลำที่เท่าไรขององค์การนาซา

7  ทะเลสาบใดในประเทศบอตสวานาที่ถือว่าเป็นทะเลสาบเกลือที่ใหญ่ที่สุดในโลก

8  ผู้ใดที่ถือว่าเป็นชาวยุโรปคนแรกที่เดินทางไปประเทศนามิเบีย

9  สามก๊ก ฉบับคนเดินดินที่ใช้นามปากกาว่า เล่า ชวน หัว เป็นผลงานของใคร

10  หยกลายเมฆ เป็นนวนิยายที่ประพันธ์โดยใคร



Cluster ID: 1
Number of questions in this cluster: 315

1  หม่อมเจ้าขจรจรัสฤทธิ์ กฤดากร ประสูติเมื่อวันที่เท่าไร

2  พระวรวงศ์เธอ พระองค์เจ้าจรูญศักดิ์กฤดากร ประสูติเมื่อวันที่เท่าไร

3  สมเด็จกรมพระนโรดม รณฤทธ

#### __Try out:__ Cluster with the different number of clusters (N_CLUSTERS) and see the result.

In [25]:
# Change the number of clusters
N_CLUSTERS = 4

In [26]:

centroids, labels = build_clusters(N_CLUSTERS)
explore_cluster(N_CLUSTERS, centroids, labels, 10)

Cluster ID: 0
Number of questions in this cluster: 539

1  บิดาของหม่อมหลวงชโยทิต กฤดากร คือใคร

2  หม่อมราชวงศ์กุลปราโมทย์ จิรประวัติ ราชสกุลเดิมคืออะไร

3  พระวรวงศ์เธอ พระองค์เจ้าชิดเชื้อพงศ์ หรือพระนามเดิมคืออะไร

4  ผู้แปลนวนิยายเรื่องสาวทรงเสน่ห์คือใคร

5  หม่อมเจ้าองค์สุดท้ายแห่งราชสกุลจิตรพงศ์คือผู้ใด

6  หม่อมเจียม คชเสนี เป็นธิดาของใคร

7  นางประเทือง เทวกุล ณ อยุธยา ใช้สกุลเดิมคืออะไร

8  สถาปนิกชาวอังกฤษที่สร้างวังจักรพงษ์ คือใคร

9  พิธีกรของรายการพินิจนครคือใคร

10  สิ่งที่ทำให้เต่าแตกต่างจากสัตว์เลื้อยคลานชนิดอื่นๆคืออะไร



Cluster ID: 1
Number of questions in this cluster: 1507

1  แกรี่ เลวิน เกิดที่เมืองอะไรในประเทศอังกฤษ

2  แกรี่ เลวิน จบปริญญาตรี ในสาขาอะไร

3  เครื่องบินขับไล่ ดัซโซลท์ มิราจ เอฟ1 ของตระกูลมิราจ ได้ประจำกองทัพฝรั่งเศสในปีใด

4  อัทธนียา เอี่ยมวสันต์ เล่นละครเรื่อง ตะวันทอแสง ที่ออกอากาศทางช่อง 7 รับบทเป็นใคร

5  บ้านตุ๊กตาได้รับรางวัลอาคารอนุรักษ์ดีเด่น จากสมาคมสถาปนิกสยาม เมื่อปีใด

6  การชกชิงแชมป์แห่งสยาม หวัง อาหะหมัดได้รับเลือกให้ชกกับ ศุข สม